The purpose of this notebook is to get the head position and direction of the ferret for each stimulus presentation.

The input data consists of two sources:
(1) Stimulus metadata after temporal alignment to both video and neural recording systems
(2) LED tracking data

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

In [7]:
data_dir = Path('/home/stephen/Github/Speaker_Grid/data/F1901_Crumble_Squid/2021-05-27_Squid_15-57')

stim_file = '2021-05-27T17-57-07_StimData_MCSVidAlign.csv'
tracking_file = '2021-05-27_SquidVid_17_56_08.csv'

stim = pd.read_csv( str(data_dir / stim_file))

print(stim.shape)

LEDs = pd.read_csv( str(data_dir / tracking_file))

print(LEDs.shape)

(829, 10)
(18345, 14)


In [25]:
join_data = pd.merge( stim, LEDs[['blue_x','red_x','blue_y','red_y']], left_on='closest_frame', right_index=True)

print(join_data.shape)

(829, 14)


### Tracking data: Problematic positions

We know that when the LEDs go out of view, the tracking code returns ridiculous values. Here we replace anything that isn't sensible with nans.

Note that y values tend to be reasonable even when x values are not (e.g. x=2703, y=123, for a 640x480 image) and so we stop the calculation using the nan on the x value.

In [26]:
join_data.loc[join_data['blue_x'] < 150, 'blue_x'] = np.nan
join_data.loc[join_data['blue_x'] > 500, 'blue_x'] = np.nan
join_data.loc[join_data['blue_y'] < 0, 'blue_y'] = np.nan
join_data.loc[join_data['blue_y'] > 480, 'blue_y'] = np.nan

join_data.loc[join_data['red_x'] < 150, 'red_x'] = np.nan
join_data.loc[join_data['red_x'] > 500, 'red_x'] = np.nan
join_data.loc[join_data['red_y'] < 0, 'red_x'] = np.nan
join_data.loc[join_data['red_y'] > 480, 'red_x'] = np.nan


Calculate head position as center between red and blue LEDs - this is limited because of the poor detection of the red LED and could be improved using only the signal from the blue LED, but since we care most about head direction and need both LEDs to calculate head angle, we'll stick with it

In [28]:
join_data['head_x'] = (join_data['blue_x'] + join_data['red_x']) / 2
join_data['head_y'] = (join_data['blue_y'] + join_data['red_y']) / 2

# Show some examples to get a feel for the data
join_data[['head_x','blue_x','red_x','head_y','blue_y','red_y']].head(n=5)


,head_x,blue_x,red_x,head_y,blue_y,red_y
0,323.310369,317.981450,328.639288,108.732587,107.364295,110.100879
1,324.682402,318.601083,330.763721,106.546165,104.032659,109.059672
2,324.526259,318.309560,330.742959,107.277053,105.433078,109.121029
3,324.755314,318.678372,330.832257,106.759202,104.321904,109.196499
4,NaN,317.878278,NaN,193.557949,107.095400,280.020499


Get head direction - here we're assuming that the red LED is on the right side of the head, and the blue LED is on the left side, such that the vector between them represents that interaural axis. In reality, there is a slight rotation in the vector that offsets it from the interaural axis, but we're ignoring that to develop the analysis quickly (and the 2D solution to this problem is inherantly bad anyway)

In [31]:
join_data['blue_zero_x'] = join_data['blue_x'] - join_data['head_x']
join_data['blue_zero_y'] = join_data['blue_y'] - join_data['head_y']

join_data['head_angle'] = np.arctan2(join_data['blue_zero_y'], join_data['blue_zero_x'])

join_data[['head_angle','blue_zero_x','blue_zero_y']].head(n=5)


,head_angle,blue_zero_x,blue_zero_y
0,-2.890255,-5.328919,-1.368292
1,-2.749660,-6.081319,-2.513506
2,-2.853243,-6.216700,-1.843976
3,-2.760162,-6.076943,-2.437297
4,NaN,NaN,-86.462549


# TO DO

### Head Stim Angles

Something we need to add is the relationship between the head location and stimulus location. However this requires a bit more information than we currently have, because stimulus locations are known in the real world (e.g. in measurements in cm relative to the grid) whereas LED positions are known only in terms of their position relative to the camera, in pixel values. 

We therefore need some form of calibration, either by mapping LED positions into the world (using some knowledge about the distance of grid features in the image), or by knowing the pixel positions of speakers. I'm going to map the centre positions of each speaker location to make the 

In [ ]:
save_file = stim


join_data.to_csv( )